In [2]:
!pip install graphframes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 374.8 kB/s eta 0:00:00a 0:00:01


In [3]:
# Cellule 1: Initialisation Spark et Chargement
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, sum as spark_sum, coalesce
from pyspark.sql.window import Window
from graphframes import GraphFrame

# Configuration Spark (pour l'environnement HDFS et GraphFrames)
spark = SparkSession.builder \
    .appName("Fraud-GraphX-Analysis-Final") \
    .config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12") \
    .getOrCreate()

# Chemin de base HDFS
hdfs_base_path = "hdfs://namenode:8020/fraud_data"

# Chargement du jeu de données original (comme confirmé précédemment)
print("--- Chargement des données originales ---")
try:
    df = spark.read.csv(
        f"{hdfs_base_path}/creditcard.csv", 
        header=True, 
        inferSchema=True
    )
    df.cache()
    print(f"Nombre total de transactions : {df.count()}")
    df.printSchema()
except Exception as e:
    print(f"ERREUR DE CHARGEMENT : {e}")
    # Ajoutez ici un chemin de secours si le HDFS n'est pas accessible

--- Chargement des données originales ---
Nombre total de transactions : 284807
root
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: dou

In [4]:
# Cellule 2/8 : Création des Vertices (Nœuds)
print("--- Création des Vertices (Clients V1) ---")

# Les Nœuds sont les identifiants V1 uniques
vertices_final = df.select(
    col("V1").alias("id"),  
    col("Class").alias("label"),
    col("Amount")
).distinct()

# La colonne 'label' doit être numérique (0 ou 1) pour les calculs.
vertices_final = vertices_final.withColumn("label", col("label").cast("int"))

# Le PageRank initial (sera réinitialisé plus tard)
vertices_final = vertices_final.withColumn("pagerank", lit(1.0))
vertices_final.cache()

print(f"Nombre de nœuds uniques (V1) : {vertices_final.count()}")
vertices_final.show(5)

--- Création des Vertices (Clients V1) ---
Nombre de nœuds uniques (V1) : 275663
+----------------+-----+------+--------+
|              id|label|Amount|pagerank|
+----------------+-----+------+--------+
|-1.2768303373631|    0| 110.4|     1.0|
|1.21205680491093|    0|  2.28|     1.0|
|1.08102680841932|    0| 17.24|     1.0|
|1.49157444507907|    0|   2.0|     1.0|
|1.09337038677875|    0|  49.9|     1.0|
+----------------+-----+------+--------+
only showing top 5 rows



In [5]:
# Cellule 3/8 : Création des Edges (Liens Frauduleux Co-occurrents)
print("--- Création des Edges (Liens Frauduleux Co-occurrents) ---")

edges = df.alias("t1").join(
    df.alias("t2"),
    (
        # 1. Les deux sont des fraudes
        (col("t1.Class") == 1) & (col("t2.Class") == 1) &
        
        # 2. Elles se produisent dans la même heure
        ((col("t1.Time") / 3600).cast("int") == (col("t2.Time") / 3600).cast("int")) &
        
        # 3. Ce sont des entités/clients différents (src != dst)
        (col("t1.V1") != col("t2.V1"))
    )
).select(
    col("t1.V1").alias("src"),
    col("t2.V1").alias("dst")
).distinct()

edges.cache()
print(f"Nombre total d'arêtes (liens de fraude) : {edges.count()}")
edges.show(5)

--- Création des Edges (Liens Frauduleux Co-occurrents) ---
Nombre total d'arêtes (liens de fraude) : 7016
+-----------------+-----------------+
|              src|              dst|
+-----------------+-----------------+
|0.314596589729515|-1.58550536691994|
|-4.72771265581559|-2.58961719821269|
|-16.5986647432584|-25.2663550194138|
|-19.8563223334433|  -27.84818067198|
|-2.78724793061533|  -27.84818067198|
+-----------------+-----------------+
only showing top 5 rows



In [6]:
# Cellule 4/8 : Création du GraphFrame et Analyse Structurale

# 1. Conversion des IDs en String (Obligatoire pour GraphFrames)
print("--- Vérification et Ajustement du Schéma pour GraphFrame ---")
vertices = vertices_final.withColumn("id", col("id").cast("string"))
edges = edges.withColumn("src", col("src").cast("string"))\
             .withColumn("dst", col("dst").cast("string"))

# 2. Création du GraphFrame
g = GraphFrame(vertices, edges)
print("\n✅ GraphFrame créé avec succès.")

# 3. Afficher les 5 nœuds les plus connectés (Degré)
try:
    print("\nTop 5 des Nœuds (V1) les plus connectés (Degré) :")
    g.degrees.orderBy(col("degree").desc()).show(5)
except Exception as e:
    print(f"Avertissement: Le calcul des degrés a échoué ({e}). Le GraphFrame n'est peut-être pas entièrement fonctionnel.")

--- Vérification et Ajustement du Schéma pour GraphFrame ---

✅ GraphFrame créé avec succès.

Top 5 des Nœuds (V1) les plus connectés (Degré) :


/usr/local/spark/python/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/usr/local/spark/python/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+-----------------+------+
|               id|degree|
+-----------------+------+
|-10.6457996485752|    84|
|-5.31417320646342|    84|
|-16.5265065691231|    84|
|-2.14441147422114|    84|
|-3.14025953779538|    84|
+-----------------+------+
only showing top 5 rows



In [7]:
# Cellule 5/8 : Algorithme des Composantes Connexes (CC)

# Définir le répertoire de checkpoint (obligatoire pour CC)
try:
    spark.sparkContext.setCheckpointDir("/tmp/gftest_checkpoints")
    print("Répertoire de checkpoint défini.")
except Exception as e:
    print(f"Avertissement: Impossible de définir le checkpoint ({e}). L'algorithme CC peut échouer.")
    
print("\n--- Calcul des Composantes Connexes ---")

cc_results = g.connectedComponents().persist()

# Analyse des Résultats
component_sizes = cc_results.groupBy("component").count().withColumnRenamed("count", "componentSize")
cc_analysis = cc_results.join(component_sizes, "component")

# Afficher les statistiques du plus grand groupe
top_component = cc_analysis.orderBy(col("componentSize").desc()).limit(1).collect()[0]
top_component_id = top_component["component"]
total_size = top_component["componentSize"]

print(f"\nStatistiques pour la Plus Grande Composante (ID: {top_component_id}):")
print(f"- Taille Totale : {total_size}")
print(f"- Composition (Label 1) : {cc_analysis.filter(col('component') == top_component_id).filter(col('label') == 1).count()} (Pureté : {cc_analysis.filter(col('component') == top_component_id).filter(col('label') == 1).count() / total_size * 100:.2f}%)")


Répertoire de checkpoint défini.

--- Calcul des Composantes Connexes ---

Statistiques pour la Plus Grande Composante (ID: 20584):
- Taille Totale : 43
- Composition (Label 1) : 43 (Pureté : 100.00%)


In [8]:
# Cellule 6/8 : Préparation des Poids PageRank (Normalisation)

print("--- Initialisation des Poids PageRank (Normalisation des Arêtes) ---")

# 1. Joindre 'edges' avec 'df' pour récupérer 'Amount'
edges_with_amount = edges.alias("e").join(
    df.select(col("V1").cast("string").alias("src_v1"), col("Amount")).distinct(), 
    col("e.src") == col("src_v1"),
    "left"
).select(col("e.src"), col("e.dst"), col("Amount")).distinct()

# 2. Calculer la somme totale des montants sortants par source
sum_weights = edges_with_amount.groupBy("src").agg(spark_sum("Amount").alias("total_outgoing_amount"))

# 3. Normalisation (Weight = Amount / Total Outgoing)
edges_with_sum = edges_with_amount.join(sum_weights, "src")
normalized_edges = edges_with_sum.withColumn(
    "weight", col("Amount") / col("total_outgoing_amount")
).select(col("src"), col("dst"), col("weight"))

# 4. Finaliser l'initialisation du PageRank (1/N)
N_vertices = vertices.count()
# Utiliser le DataFrame 'vertices' ajusté des étapes précédentes
vertices = vertices.withColumn("pagerank", lit(1.0 / N_vertices)).cache()

print(f"Le DataFrame 'normalized_edges' est prêt avec {normalized_edges.count()} arêtes.")
print(f"PageRank initial (1/N) : {1.0 / N_vertices:.8f}")

--- Initialisation des Poids PageRank (Normalisation des Arêtes) ---
Le DataFrame 'normalized_edges' est prêt avec 7016 arêtes.
PageRank initial (1/N) : 0.00000363


In [9]:
# 🔹 Cellule 7/8 : Algorithme PageRank (Simulation Native PySpark)

from pyspark.sql.functions import sum, lit, col, coalesce # S'assurer que les fonctions sont bien importées

MAX_ITER = 5
RESET_PROBABILITY = 0.15 

print(f"\n--- PageRank Simulé ({MAX_ITER} itérations) ---")

# 'vertices' contient id, label, Amount, pagerank initial. On commence avec id, label, pagerank
current_vertices = vertices.select("id", "label", "pagerank").cache()

for i in range(MAX_ITER):
    # 1. Calcul du Score de contribution (pr * weight)
    contributions = current_vertices.join(
        normalized_edges, current_vertices.id == normalized_edges.src
    ).withColumn(
        "contribution", col("pagerank") * col("weight")
    ).select(col("dst").alias("id"), "contribution")
    
    # 2. Agrégation des contributions
    new_pageranks = contributions.groupBy("id").agg(sum("contribution").alias("sum_contribution"))
    
    # 3. Application de la formule PageRank
    current_vertices = current_vertices.drop("pagerank")\
                                       .join(new_pageranks, "id", "left_outer")\
                                       .withColumn(
                                           "sum_contribution_clean", 
                                           coalesce(col("sum_contribution"), lit(0))
                                       )
    
    N = float(current_vertices.count())
    
    # PR(new) = (1 - alpha) * PR(contribution) + alpha / N
    current_vertices = current_vertices.withColumn(
        "pagerank", 
        lit(1.0 - RESET_PROBABILITY) * col("sum_contribution_clean") + lit(RESET_PROBABILITY / N)
    ).select("id", "label", "pagerank").cache()
    
    print(f"Itération {i+1} complétée.")
    
# Réintégrer la colonne Amount pour l'affichage final, en sélectionnant et renommant les colonnes clairement
# *********************************** CORRECTION ICI *********************************
results = current_vertices.join(
    vertices.select(
        "id", 
        "Amount", 
        col("label").alias("original_label") # Renommer pour éviter l'ambiguïté
    ), 
    "id", 
    "left"
).drop("label").withColumnRenamed("original_label", "label") # Supprimer le label ambigu et renommer le bon

results = results.orderBy(col("pagerank").desc())
print("\nPageRank Simulation terminée. Résultats prêts pour l'affichage.")


--- PageRank Simulé (5 itérations) ---
Itération 1 complétée.
Itération 2 complétée.
Itération 3 complétée.
Itération 4 complétée.
Itération 5 complétée.

PageRank Simulation terminée. Résultats prêts pour l'affichage.


In [12]:
# Cellule 8/8 : Affichage et Interprétation des Résultats Finaux

# Renommage de la colonne pour la clarté
final_results = results.withColumnRenamed("pagerank", "PR_Score")

# 1. Top 10 des Hubs Potentiels
print("\n--- Top 10 des Clients/Cartes (V1) ayant le PageRank le plus élevé (Hubs potentiels) ---")
final_results.select("id", "label", "Amount", "PR_Score").show(10, truncate=False)

# 2. Cas potentiels de Mules (transactions légitimes avec un PageRank élevé)
print("\n--- Top 5 des Transactions Légitimes (Label 0) avec PageRank élevé (Mules potentielles) ---")
final_results.filter(col("label") == 0) \
    .orderBy(col("PR_Score").desc()) \
    .show(5, truncate=False)

# 3. Conclusion des résultats
print("\n--- Interprétation Finale ---")
print(f"CC : {cc_analysis.select('componentSize').distinct().count()} groupes de fraude détectés.")
print("PageRank : Les scores PageRank permettent de classer le risque au sein du groupe principal.")
print("La combinaison CC + PageRank fournit une vision complète de l'organisation de la fraude.")


--- Top 10 des Clients/Cartes (V1) ayant le PageRank le plus élevé (Hubs potentiels) ---
+-----------------+-----+------+--------------------+
|id               |label|Amount|PR_Score            |
+-----------------+-----+------+--------------------+
|-7.50392623748137|1    |12.31 |3.627617779680262E-6|
|-2.4340041522657 |1    |362.55|3.627617779680262E-6|
|-2.98646550822273|1    |1.79  |3.627617779680262E-6|
|-17.5375916846763|1    |9.82  |3.627617779680262E-6|
|1.22761441351286 |1    |98.01 |3.627617779680262E-6|
|1.37855899734127 |1    |0.76  |3.627617779680262E-6|
|0.432554461820961|1    |1.0   |3.627617779680262E-6|
|-2.78386548658584|1    |156.0 |3.627617779680262E-6|
|0.467991939825149|1    |120.54|3.627617779680262E-6|
|1.24384844934819 |1    |1.0   |3.627617779680262E-6|
+-----------------+-----+------+--------------------+
only showing top 10 rows


--- Top 5 des Transactions Légitimes (Label 0) avec PageRank élevé (Mules potentielles) ---
+------------------+---------------